<a href="https://colab.research.google.com/github/JustNoel05/POO-25A/blob/main/ProyectoFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
classDiagram
    class BirthdayManager {
        -birthdays_file: str
        -messages_file: str
        -birthdays: DataFrame
        -messages: DataFrame
        +__init__(birthdays_file: str, messages_file: str)
        +load_birthdays(): DataFrame
        +load_messages(): DataFrame
        +save_birthdays(): None
        +save_messages(): None
        +add_birthday(name: str, date_input, email: str): bool
        +add_message(message_name: str, content: str): bool
        +upcoming_birthdays(days: int=30): DataFrame
        +send_birthday_email(receiver_email: str, message_content: str): bool
    }

    class EmailSender {
        +send_email(smtp_server: str, port: int, username: str, password: str, to_email: str, subject: str, message: str): bool
    }

    BirthdayManager --> EmailSender: Usa para enviar correos

In [1]:
%%writefile app.py
import pandas as pd
from datetime import datetime, timedelta, date
import streamlit as st
import os
import smtplib
import ssl
import random
from email.mime.text import MIMEText

class BirthdayManager:
    def __init__(self, birthdays_file='birthdays.csv', messages_file='messages.csv'):
        self.birthdays_file = birthdays_file
        self.messages_file = messages_file

        if not os.path.exists(self.birthdays_file):
            pd.DataFrame(columns=['name', 'date', 'email']).to_csv(self.birthdays_file, index=False)
        if not os.path.exists(self.messages_file):
            pd.DataFrame(columns=['message_name', 'content']).to_csv(self.messages_file, index=False)

        self.birthdays = self.load_birthdays()
        self.messages = self.load_messages()

    def load_birthdays(self):
        try:
            df = pd.read_csv(self.birthdays_file)
            df['date'] = pd.to_datetime(df['date'], errors='coerce').dt.date
            df = df.dropna(subset=['date'])
            return df
        except Exception as e:
            st.error(f"Error al cargar cumpleaños: {str(e)}")
            return pd.DataFrame(columns=['name', 'date', 'email'])

    def load_messages(self):
        try:
            return pd.read_csv(self.messages_file)
        except Exception as e:
            st.error(f"Error al cargar mensajes: {str(e)}")
            return pd.DataFrame(columns=['message_name', 'content'])

    def save_birthdays(self):
        self.birthdays.to_csv(self.birthdays_file, index=False)

    def save_messages(self):
        self.messages.to_csv(self.messages_file, index=False)

    def add_birthday(self, name, date_input, email):
        try:
            if isinstance(date_input, (pd.Timestamp, datetime)):
                date_input = date_input.date()
            elif isinstance(date_input, str):
                date_input = datetime.strptime(date_input, '%Y-%m-%d').date()

            new_entry = pd.DataFrame({
                'name': [name],
                'date': [date_input],
                'email': [email]
            })
            self.birthdays = pd.concat([self.birthdays, new_entry], ignore_index=True)
            self.save_birthdays()
            return True
        except Exception as e:
            st.error(f"Error al agregar cumpleaños: {str(e)}")
            return False

    def add_message(self, message_name, content):
        try:
            new_entry = pd.DataFrame({
                'message_name': [message_name],
                'content': [content]
            })
            self.messages = pd.concat([self.messages, new_entry], ignore_index=True)
            self.save_messages()
            return True
        except Exception as e:
            st.error(f"Error al agregar mensaje: {str(e)}")
            return False

    def upcoming_birthdays(self, days=30):
        try:
            today = date.today()
            end_date = today + timedelta(days=days)

            def calculate_next_birthday(birth_date):
                if not isinstance(birth_date, date):
                    if isinstance(birth_date, str):
                        birth_date = datetime.strptime(birth_date, '%Y-%m-%d').date()
                    else:
                        birth_date = pd.to_datetime(birth_date).date()

                next_bday = date(today.year, birth_date.month, birth_date.day)
                if next_bday < today:
                    next_bday = date(today.year + 1, birth_date.month, birth_date.day)
                return next_bday

            self.birthdays['next_birthday'] = self.birthdays['date'].apply(calculate_next_birthday)

            upcoming = self.birthdays[
                (self.birthdays['next_birthday'] >= today) &
                (self.birthdays['next_birthday'] <= end_date)
            ]

            return upcoming[['name', 'next_birthday', 'email']].sort_values(by='next_birthday')
        except Exception as e:
            st.error(f"Error al calcular próximos cumpleaños: {str(e)}")
            return pd.DataFrame(columns=['name', 'next_birthday', 'email'])

    def send_birthday_email(self, receiver_email, message_content):
        port = 587
        smtp_server = "smtp.gmail.com"
        sender_email = "fvpootisthegoat@gmail.com"
        password = "oadg lhku dmkq aukq"

        message = MIMEText(message_content)
        message['Subject'] = "¡Feliz Cumpleaños!"
        message['From'] = sender_email
        message['To'] = receiver_email

        try:
            context = ssl.create_default_context()
            with smtplib.SMTP(smtp_server, port) as server:
                server.ehlo()
                server.starttls(context=context)
                server.ehlo()
                server.login(sender_email, password)
                server.sendmail(sender_email, receiver_email, message.as_string())
            return True
        except Exception as e:
            st.error(f"Error al enviar correo: {str(e)}")
            return False

def main():
    st.set_page_config(page_title="Gestor de Cumpleaños", layout="wide")
    manager = BirthdayManager()

    st.title("🥳 Gestor de Cumpleaños todo pro 🥳")

    tab1, tab2, tab3, tab4 = st.tabs(["Registro", "Próximos", "Mensajes", "Enviar"])

    with tab1:
        st.header("Registrar Cumpleaños")
        name = st.text_input("Nombre completo")
        date_input = st.date_input(
            "Fecha de nacimiento",
            min_value=date(1900, 1, 1),
            max_value=date.today() + timedelta(days=365),
            value=date(2000, 1, 1)
        )
        email = st.text_input("Correo electrónico")

        if st.button("Guardar registro"):
            if manager.add_birthday(name, date_input, email):
                st.success(f"✅ Cumpleaños de {name} registrado correctamente!")

    with tab2:
        st.header("Próximos Cumpleaños")
        days = st.slider("Mostrar próximos días", 1, 365, 30)
        upcoming = manager.upcoming_birthdays(days)

        if not upcoming.empty:
            st.dataframe(upcoming)
        else:
            st.info("No hay cumpleaños próximos en este período")

    with tab3:
        st.header("Gestión de Mensajes")
        message_name = st.text_input("Nombre del mensaje")
        message_content = st.text_area("Contenido del mensaje", height=150)

        col1, col2 = st.columns(2)
        with col1:
            if st.button("Guardar mensaje"):
                if manager.add_message(message_name, message_content):
                    st.success("Mensaje guardado exitosamente!")

        with col2:
            if st.button("Mostrar mensajes aleatorio"):
                if not manager.messages.empty:
                    random_msg = random.choice(manager.messages['content'].values)
                    st.text_area("Mensaje aleatorio", value=random_msg, height=150)
                else:
                    st.warning("No hay mensajes guardados")

        if not manager.messages.empty:
            st.subheader("Mensajes guardados")
            st.dataframe(manager.messages)

    with tab4:
        st.header("Enviar Felicitaciones")

        today_birthdays = manager.upcoming_birthdays(1)

        if not today_birthdays.empty:
            st.subheader("Cumpleañeros de hoy")

            for _, row in today_birthdays.iterrows():
                with st.expander(f"Enviar a {row['name']}"):
                    selected_message = st.selectbox(
                        f"Seleccionar mensaje para {row['name']}",
                        options=[""] + list(manager.messages['message_name']),
                        key=f"msg_{row['name']}"
                    )

                    if selected_message:
                        message_content = manager.messages[manager.messages['message_name'] == selected_message]['content'].values[0]
                    else:
                        message_content = st.text_area(f"Mensaje personalizado para {row['name']}", height=150)

                    if st.button(f"Enviar a {row['name']}"):
                        if manager.send_birthday_email(row['email'], message_content):
                            st.success(f"Correo enviado a {row['name']}!")
        else:
            st.info("No hay cumpleaños hoy para felicitar")

if __name__ == "__main__":
    main()

Writing app.py


In [2]:
!pip install streamlit pandas
!npm install localtunnel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.6 MB/s eta 0:00:00
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹
added 22 packages in 3s
⠹
⠹3 packages are looking for funding
⠹  run `npm fund` for details
⠹

In [ ]:
!streamlit run app.py --server.port 8501 & npx localtunnel --port 8501

⠙⠹⠸

your url is: https://common-signs-scream.loca.lt

  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.16.45.238:8501

